Step 1: Install Required Libraries

In [1]:
!pip install -q langchain langchain-community pypdf sentence-transformers faiss-cpu transformers torch accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

Step 2: Upload PDF Research Papers

In [2]:
from google.colab import files

uploaded = files.upload()
pdf_paths = list(uploaded.keys())

Saving 1706.03762v7.pdf to 1706.03762v7.pdf
Saving 2005.11401v4.pdf to 2005.11401v4.pdf
Saving 2005.14165v4.pdf to 2005.14165v4.pdf


Step 3: Load & Chunk the Documents

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

all_docs = []

for pdf in pdf_paths:
    loader = PyPDFLoader(pdf)
    docs = loader.load_and_split()
    all_docs.extend(docs)

print(f"Loaded {len(all_docs)} document chunks.")

Loaded 136 document chunks.


Step 4: Create Vector Store Using Embeddings

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(all_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})  # Retrieve top 4 chunks

<ipython-input-4-2903540997>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or data

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Step 5: Load a Local LLM for Answer Generation

Let’s use Hugging Face's T5 model for generation. For better performance, consider google/flan-t5-large or mistralai/Mistral-7B-Instruct.

In [5]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
<ipython-input-5-2859017371>:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


Step 6: Set Up Retrieval-Augmented QA Chain with Source Attribution

In [6]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

qa_prompt = PromptTemplate.from_template("""
You are an AI assistant helping answer technical questions about AI research papers.
Use the following pieces of context to answer the question. If you don't know the answer, say so clearly.

Context:
{context}

Question:
{question}

Answer:
""")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": qa_prompt},
    return_source_documents=True
)

Step 7: Ask Questions

In [7]:
questions = [
    "What are the main components of a RAG model, and how do they interact?",
    "What are the two sub-layers in each encoder layer of the Transformer model?",
    "Explain how positional encoding is implemented in Transformers and why it is necessary.",
    "Describe the concept of multi-head attention in the Transformer architecture. Why is it beneficial?",
    "What is few-shot learning, and how does GPT-3 implement it during inference?"
]

for q in questions:
    result = qa_chain.invoke(q)
    print("❓ Question:", q)
    print("✅ Answer:", result["result"])

    # Show sources
    print("\n📄 Sources:")
    for i, doc in enumerate(result["source_documents"]):
        print(f"\nSource {i+1} (Page {doc.metadata['page']}):")
        print(doc.page_content[:300] + "...")
    print("\n" + "-"*80 + "\n")

Token indices sequence length is longer than the specified maximum sequence length for this model (1534 > 512). Running this sequence through the model will result in indexing errors
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


❓ Question: What are the main components of a RAG model, and how do they interact?
✅ Answer: retriever p (z |x ) with parameters  that returns (top-K truncated) distributions over text passages given a query x and (ii) a generator p (y i |x,z,y 1:i 1) parametrized

📄 Sources:

Source 1 (Page 1):
the non-parametric memory can be replaced to update the models’ knowledge as the world changes.1
2 Methods
We explore RAG models, which use the input sequencex to retrieve text documents z and use them
as additional context when generating the target sequence y . As shown in Figure 1, our models
lev...

Source 2 (Page 17):
documents for questions that are less likely to beneﬁt from retrieval, suggesting that null document
mechanisms may not be necessary for RAG.
G Parameters
Our RAG models contain the trainable parameters for the BERT-base query and document encoder of
DPR, with 110M parameters each (although we do no...

Source 3 (Page 5):
to guide the generation, drawing out speciﬁc knowledge

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


❓ Question: What are the two sub-layers in each encoder layer of the Transformer model?
✅ Answer: The first is a multi-head self-attention mechanism, and the second is a simple, position- wise fully connected feed-forward network

📄 Sources:

Source 1 (Page 2):
Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder...

Source 2 (Page 4):
output values. These are concatenated and once again projected, resulting in the final values, as
depicted in Figure 2.
Multi-head attention allows the model to jointly attend to information from different representation
subspaces at different positions. With a single attention head, averaging inhib...

Source 3 (Page 5):
Table 1: Maximum path lengths, per-layer complexity and minimum number of sequential opera

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


❓ Question: Explain how positional encoding is implemented in Transformers and why it is necessary.
✅ Answer: We employ a residual connection [11] around each of the two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension dmodel = 512. Decoder: The decoder is also composed of a stack of N = 6identical layers. In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections around each of the sub-layers, followed by layer normalization. We also modify the self-attention

📄 Sources:

Source 1 (Page 2):
Figure 1: The Transformer - model architecture.
The Transformer

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


❓ Question: Describe the concept of multi-head attention in the Transformer architecture. Why is it beneficial?
✅ Answer: The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all of the keys, values and queries come from the same place, in this case, the output of the previous layer in the encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder. • Similarly, self-attention layers in the decoder allow each position in the decoder to attend to all positions in the decoder up

📄 Sources:

Source 1 (P

KeyboardInterrupt: 

Step 5: Configure Gemini API

In [22]:
# In the part of your code where you use the Gemini API, ensure you get the key:
from google.colab import userdata
import google.generativeai as genai # Import the Gemini API library

# Access the stored API key
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure the Gemini API
genai.configure(api_key=GOOGLE_API_KEY)

# Now you can use genai.GenerativeModel etc. as intended.
model = genai.GenerativeModel('gemini-pro') # Example model initialization

Step 6: Define Prompt Template

In [23]:
from langchain.prompts import PromptTemplate

rag_prompt = PromptTemplate.from_template("""
You are an AI assistant helping answer technical questions about AI research.
Use the following pieces of context from research papers to answer the question.
If the answer is not present in the context, say so clearly.

Context:
{context}

Question:
{question}

Answer:
""")

Step 7: Custom RAG Function Using Gemini

In [24]:
def rag_qa_with_gemini(question):
    # Retrieve relevant documents
    docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in docs])

    # Build prompt
    prompt = rag_prompt.format(context=context, question=question)

    # Ask Gemini
    response = model.generate_content(
        prompt,
        safety_settings=[
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}
        ]
    )

    return {
        "result": response.text,
        "source_documents": docs
    }

Step 8: Test with Sample Questions

In [26]:
# List available models to confirm the name and supported methods
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thi

In [27]:
model = genai.GenerativeModel('gemini-pro')

In [28]:
model = genai.GenerativeModel('models/gemini-pro')

In [32]:
# In the part of your code where you use the Gemini API, ensure you get the key:
from google.colab import userdata
import google.generativeai as genai # Import the Gemini API library

# Access the stored API key
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure the Gemini API
genai.configure(api_key=GOOGLE_API_KEY)

# Now you can use genai.GenerativeModel etc. as intended.
# Use the full model name 'models/gemini-pro' as suggested by the error structure
model = genai.GenerativeModel('models/gemini-pro') # Example model initialization

In [33]:
model = genai.GenerativeModel('gemini-pro')

In [34]:
def rag_qa_with_gemini(question):
    docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = rag_prompt.format(context=context, question=question)

    try:
        response = model.generate_content(
            prompt,
            safety_settings=[
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}
            ]
        )
        return {
            "result": response.text,
            "source_documents": docs
        }
    except Exception as e:
        return {
            "result": f"⚠️ Error generating answer: {str(e)}",
            "source_documents": docs
        }

In [35]:
questions = [
    "What are the main components of a RAG model, and how do they interact?",
    "What are the two sub-layers in each encoder layer of the Transformer model?",
    "Explain how positional encoding is implemented in Transformers and why it is necessary.",
    "Describe the concept of multi-head attention in the Transformer architecture. Why is it beneficial?",
    "What is few-shot learning, and how does GPT-3 implement it during inference?"
]

for q in questions:
    result = rag_qa_with_gemini(q)
    print("❓ Question:", q)
    print("✅ Answer:", result["result"])

    print("\n📄 Sources:")
    for i, doc in enumerate(result["source_documents"]):
        print(f"\nSource {i+1} (Page {doc.metadata['page']}):")
        print(doc.page_content[:300] + "...")
    print("\n" + "-"*80 + "\n")

❓ Question: What are the main components of a RAG model, and how do they interact?
✅ Answer: ⚠️ Error generating answer: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

📄 Sources:

Source 1 (Page 1):
the non-parametric memory can be replaced to update the models’ knowledge as the world changes.1
2 Methods
We explore RAG models, which use the input sequencex to retrieve text documents z and use them
as additional context when generating the target sequence y . As shown in Figure 1, our models
lev...

Source 2 (Page 17):
documents for questions that are less likely to beneﬁt from retrieval, suggesting that null document
mechanisms may not be necessary for RAG.
G Parameters
Our RAG models contain the trainable parameters for the BERT-ba

❓ Question: What are the two sub-layers in each encoder layer of the Transformer model?
✅ Answer: ⚠️ Error generating answer: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

📄 Sources:

Source 1 (Page 2):
Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder...

Source 2 (Page 4):
output values. These are concatenated and once again projected, resulting in the final values, as
depicted in Figure 2.
Multi-head attention allows the model to jointly attend to information from different represen

❓ Question: Explain how positional encoding is implemented in Transformers and why it is necessary.
✅ Answer: ⚠️ Error generating answer: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

📄 Sources:

Source 1 (Page 2):
Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder...

Source 2 (Page 4):
output values. These are concatenated and once again projected, resulting in the final values, as
depicted in Figure 2.
Multi-head attention allows the model to jointly attend to information from differ

❓ Question: Describe the concept of multi-head attention in the Transformer architecture. Why is it beneficial?
✅ Answer: ⚠️ Error generating answer: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

📄 Sources:

Source 1 (Page 4):
output values. These are concatenated and once again projected, resulting in the final values, as
depicted in Figure 2.
Multi-head attention allows the model to jointly attend to information from different representation
subspaces at different positions. With a single attention head, averaging inhib...

Source 2 (Page 2):
Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and

❓ Question: What is few-shot learning, and how does GPT-3 implement it during inference?
✅ Answer: ⚠️ Error generating answer: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

📄 Sources:

Source 1 (Page 5):
completion, and then one ﬁnal example of context, with the model expected to provide the completion. We
typically set Kin the range of 10 to 100 as this is how many examples can ﬁt in the model’s context window
(nctx = 2048). The main advantages of few-shot are a major reduction in the need for task...

Source 2 (Page 12):
in the few-shot setting. This compares to 37.4% for ﬁne-tuned T5-11B, and 44.7% for ﬁne-tuned T5-11B+SSM,
which uses a Q&A-speciﬁc pre-training procedure. GPT-3 in the few-shot setting approaches the performance 